<a href="https://colab.research.google.com/github/Sowmya-2348562/LLM/blob/main/Sowmya_562_LLM_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [8]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the pre-trained model and tokenizer
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


In [9]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer


In [10]:
context = "Paris is the capital and most populous city of France. The city is a major European hub for art, fashion, and culture."
question = "What is the capital of France?"
answer = answer_question(question, context)
print("Question:", question)
print("Answer:", answer)


Question: What is the capital of France?
Answer: paris


In [17]:
# Different prompts to test the impact on the model output
prompts = [
    "What is the capital of France?",
    "Can you tell me the capital city of France?",
    "Which city is the capital of France?"
]

context = "Paris is the capital and most populous city of France. The city is a major European hub for art, fashion, and culture."

# Function to test different prompts
def test_prompts(prompts, context):
    results = {}
    for prompt in prompts:
        answer = answer_question(prompt, context)
        results[prompt] = answer
    return results

# Test the prompts
results = test_prompts(prompts, context)
for prompt, answer in results.items():
    print(f"Prompt: {prompt}")
    print(f"Answer: {answer}\n")


Prompt: What is the capital of France?
Answer: paris

Prompt: Can you tell me the capital city of France?
Answer: paris

Prompt: Which city is the capital of France?
Answer: paris



### Prompt Engineering Insights

**Prompt 1:** "What is the capital of France?"
**Output:** "Paris"

**Prompt 2:** "Can you tell me the capital city of France?"
**Output:** "Paris"

**Prompt 3:** "Which city is the capital of France?"
**Output:** "Paris"


In [12]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the pre-trained model and tokenizer
model_name = "distilbert-base-uncased-distilled-squad"  # You can choose other models as well
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Function to answer questions
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

# Define the context
contexts = {
    "largest_planet": "Jupiter is the largest planet in our solar system.",
    "relativity": "The theory of relativity, developed by Albert Einstein, revolutionized our understanding of space, time, and gravity.",
    "mockingbird": "'To Kill a Mockingbird' is a novel by Harper Lee."
}

# Define the prompts
prompts = {
    "largest_planet": "What is the largest planet in our solar system?",
    "relativity": "Explain the theory of relativity in simple terms.",
    "mockingbird": "Who wrote 'To Kill a Mockingbird'?"
}

# Get responses from your model
responses = {}
for key in prompts:
    responses[key] = answer_question(prompts[key], contexts[key])

# Print the responses
for key in prompts:
    print(f"Prompt: {prompts[key]}")
    print(f"Answer: {responses[key]}\n")


Prompt: What is the largest planet in our solar system?
Answer: jupiter

Prompt: Explain the theory of relativity in simple terms.
Answer: the theory of relativity, developed by albert einstein, revolutionized our understanding of space, time, and gravity

Prompt: Who wrote 'To Kill a Mockingbird'?
Answer: harper lee



MODEL - CHATGPT 3.5

1. **What is the largest planet in our solar system?**
   - The largest planet in our solar system is Jupiter.

2. **Explain the theory of relativity in simple terms:**
   - The theory of relativity, developed by Albert Einstein, is a fundamental theory in physics that explains how space, time, and gravity work together. It has two main parts:
     - **Special Relativity:** Deals with objects moving at constant speeds relative to each other and describes how time and space are affected.
     - **General Relativity:** Describes gravity as the curvature of space and time caused by mass and energy.

3. **Who wrote 'To Kill a Mockingbird'?**
   - 'To Kill a Mockingbird' is a novel written by Harper Lee.

MODEL 2 : Gemini AI :

The largest planet in our solar system is Jupiter.


The theory of relativity is a complex concept, but in simple terms, it suggests that space and time are not fixed but are relative to the observer and the influence of gravity. Imagine a trampoline - a heavy object like a bowling ball will warp the fabric of the trampoline more than a marble, similar to how massive objects like stars warp spacetime.


'To Kill a Mockingbird' was written by Harper Lee.

Inferences
Prompt Engineering and Optimization:

Prompt Definition: The way prompts are defined plays a crucial role in determining the quality and relevance of the model's responses. Optimizing prompts involves crafting them in a way that effectively guides the model towards generating accurate and informative answers.
Impact of Prompt Formulation: Different formulations of prompts can lead to varied outputs from the models. For example, a more specific prompt might elicit a more detailed response, while a broader prompt might yield a general answer. Understanding this impact helps in refining prompts to suit specific use cases or query types.

inference:
All models correctly identify Jupiter as the largest planet in our solar system, indicating strong factual knowledge.

ChatGPT 3.5 provides a more detailed explanation of the theory of relativity compared to My Model and Gemini AI, breaking it down into its components and offering a comprehensive understanding.

ChatGPT 3.5 excels in natural language generation, presenting complex ideas like the theory of relativity in an engaging and understandable manner.

All models consistently attribute 'To Kill a Mockingbird' to Harper Lee, demonstrating accurate factual retrieval and contextual understanding.

The impact of prompt formulation is evident, with more specific prompts leading to more detailed responses.

ChatGPT 3.5's ability to provide detailed and natural explanations makes it suitable for educational and informational purposes where clarity and depth are important.

While all models perform well in factual retrieval, ChatGPT 3.5 stands out in providing detailed and natural explanations, showcasing strength in handling nuanced prompts and delivering engaging responses suitable for a broad range of applications.